# Observation Planning

Questions to ask when planning a photometric observing session ("run"):

1. __Where is my target?__
2. __When can I observe my target?__
3. __How do I know when I've found my target (make a finder chart)?__
3. How long do I need to observe?
4. How will I calibrate my data?

## Where is my target? 

Let's pick a few exciting targets to look at.  These are gigantic evolved stars, Luminous Blue Variables and Yellow Hypergiants and Supergiants:

1. P Cygni
2. Rho Cassiopeia
3. HD 168607
4. Polaris

To find where these sources are on the sky, we can query a few databases, including the SESAME name resolver service (http://cds.u-strasbg.fr/cgi-bin/Sesame) and the SIMBAD service.  We can do this using `astropy`, the main library for astronomical analysis.


In [ ]:
from astropy import coordinates, units

In [ ]:
# this uses SESAME
pcyg_coord = coordinates.SkyCoord.from_name('P Cygni')
pcyg_coord

We can display this in other ways too:

In [ ]:
pcyg_coord.to_string('hmsdms')

The SIMBAD service gives us a little additional information:

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
pcyg_simbad = Simbad.query_object('P Cygni')
pcyg_simbad

We'll just use SESAME for the others, though.  Fill in the SESAME approach for `rhocas` (Rho Cassiopeia), `hd168607` (HD 168607), and `polaris` ("Polaris"):

(`coordinates.SkyCoord.from_name` uses SESAME to obtain the object's coordinates)

In [ ]:
# replace this cell with your answer
# the variable names should be those used in the next cell

In [ ]:
print(pcyg_coord.to_string('hmsdms'))
print(rhocas_coord.to_string('hmsdms'))
print(hd168607_coord.to_string('hmsdms'))
print(polaris_coord.to_string('hmsdms'))

## When can I observe my target?

Next, knowing where these are on the sky, we can ask what times of year they are up at night.

We first have to define where _we_ are by setting the observatory location.

In [ ]:
from astroplan import Observer
from astropy import units as u # shortcut

In [ ]:
# sometimes we have to update the database; if that's necessary, a message will pop up telling us
from astropy.utils import iers
iers.conf.IERS_A_URL = 'ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
iers.conf.IERS_A_URL_MIRROR = 'https://datacenter.iers.org/data/9/finals2000A.all'
from astroplan import download_IERS_A
download_IERS_A()

In [ ]:
CTO = Observer(location=coordinates.EarthLocation(lat=29.643018, lon=-82.349004*u.deg, height=31*u.m),
               timezone='EST',
               name='University of Florida Campus Teaching Observatory',
              )
CTO

Before we go on, we can use some information we already have in hand to determine approximately when we can observe these objects.  It is important to do this sort of qualitative 'sanity check' to ensure that the numbers coming from the computer are reasonable.

The March equinox defines the zero-point of the Right Ascension coordinate, which means that 12h RA goes overhead at midnight (0h RA goes overhead at noon).

There are 24 hours in a day and 12 months in a year.  By how many hours does the sky rotate each month?

A: <your answer here>

How many degrees?

A: <your answer here>

Given the above information, answer the following questions:

1. What is the Right Ascension of the Sun in the March (vernal) equinox?

A: <your answer here>

2. What is the RA of the Sun at the September (autumnal) equinox?

A: <your answer here>

3. During what season or month do our targets reach zenith at midnight? (one answer per target)

A:
* P Cyg: 
* Rho Cas: 
* HD 168607: 
* Polaris: 

Having done these checks, we can proceed to get more precise answers.

We create `target` objects to observe at different times.

In [ ]:
from astroplan import FixedTarget

In [ ]:
pcyg_target = FixedTarget(pcyg_coord, name="P Cygni")
rhocas_target = FixedTarget(rhocas_coord, name="Rho Cas")
hd168607_target = FixedTarget(hd168607_coord, name="HD 168607")
polaris_target = FixedTarget(polaris_coord, name="Polaris")
pcyg_target, rhocas_target, hd168607_target, polaris_target

To determine the observability of an object, we ask whether the object is "up" (in the sky, as opposed to underfoot), and whether it is night time.

What time are you working on this notebook?  Is it night?  Are the target sources "up"?  (answer these before filling out the code)

<your answer here>

Since the answers to all of our questions depend on both the time of day and the time of year, we need to define our time.

In [ ]:
from astropy.time import Time

In [ ]:
now = Time.now()
now

Are any of these sources "up"?

In [ ]:
CTO.target_is_up(now, [pcyg_target, rhocas_target, hd168607_target, polaris_target])

At least some are "up" at the time of writing.

OK, but is it night?

In [ ]:
CTO.is_night(now)

For planning purposes, we care more about what the future holds and how we're going to schedule our time, so we want to know when sunrise and sunset are.  We use `now` to tell the software that we care about sunrise and sunset _today_; we can put in future days to find out how sunrise/sunset times change.

In [ ]:
CTO.sun_set_time(now), CTO.sun_rise_time(now)

These aren't formatted in a very familiar way, so we can use a standard format - e.g., ISO, the International Standards Organization format - to show them more nicely:

In [ ]:
CTO.sun_set_time(now).iso, CTO.sun_rise_time(now).iso

Those times look a bit funny because they're in UTC, Universal Coordinated Time.

Time zones are tricky, so we rely on libraries to help us with them.

In [ ]:
now.to_datetime(timezone=CTO.timezone).isoformat()

That's a pretty verbose way to ask for and display this, so we can define a shortcut to get the time of day in the Eastern time zone:

In [ ]:
def eastern(time):
    est = time.to_datetime(timezone=CTO.timezone)
    
    return est.strftime('%H:%M:%S')

In [ ]:
(eastern(CTO.sun_set_time(now)), eastern(CTO.sun_rise_time(now)))

Now you can see that sunset is around 5:30 PM and sunrise is around 7:30 AM.  Note that we have set first, rise second, because we care about the interval between the sun going down and coming back up again, not the daytime.

We need to know the sun's rise and set time to avoid it, but we also can ask when our sources rise and set.

In [ ]:
(eastern(CTO.target_rise_time(time=now, target=pcyg_target)),
 eastern(CTO.target_set_time(time=now, target=pcyg_target)))

In [ ]:
(eastern(CTO.target_rise_time(time=now, target=rhocas_target)),
 eastern(CTO.target_set_time(time=now, target=rhocas_target)))

In [ ]:
(eastern(CTO.target_rise_time(time=now, target=hd168607_target)),
 eastern(CTO.target_set_time(time=now, target=hd168607_target)))

In [ ]:
### Note: this will fail!  "Comment it out" and move on
((CTO.target_rise_time(time=now, target=polaris_target)),
 (CTO.target_set_time(time=now, target=polaris_target)))

What's going on with that last one?  Since Polaris is the North Star, it is always observable from any northern observatory.  It has no rise and set time!

We can visualize the "up time" for the targets using an _airmass plot_, which plots the airmass versus time of night.

[Airmass](https://en.wikipedia.org/wiki/Air_mass_(astronomy)) is the path length of air through which we're observing, and it is usually approximated as secant(zenith angle) and measured relative to zenith.


1 airmass is straight up, and is the minimum amount of atmosphere through which you can observe.

Generally, it is best to observe at airmass < 2 when possible.  Airmass up to ~3 can be useful.  Beyond that, serious complications tend to arise.

In [ ]:
from astroplan.plots import plot_airmass 
import matplotlib.pyplot as plt 

plot_airmass([hd168607_target, pcyg_target, rhocas_target, polaris_target],
             observer=CTO,
             time=now,
             brightness_shading=True)

plt.legend(loc='best')
plt.show() 

Note that the X-axis is in Universal Coordinated Time (UTC).  If you want the plots in local time, you can request that the local timezone be used (but only for certain versions of astroplan):

In [ ]:
# if this fails, comment it out and continue
plot_airmass([hd168607_target, pcyg_target, rhocas_target, polaris_target],
             observer=CTO,
             time=now.to_datetime(timezone=CTO.timezone),
             use_local_tz=True,
             brightness_shading=True)

plt.legend(loc='best')
plt.show() 

Answer some questions:

Which sources are up at night?  For how long is each source observable at airmass $\lesssim2$?  (Polaris is close enough to 2 to consider it observable)

## How do I know when I've found my target? Make a finder chart

We need to know how to tell we're 'on target' when we've pointed our telescope at the right part of the sky.  With binoculars and other very wide field-of-view instruments, we can use known asterisms or constellations to navigate by, but with narrow field-of-view professional telescopes, we have to use fainter stars to aid our pointing.

We can obtain finder charts from the many public astronomical archives, like the Digitized Sky Survey, which carefully logged the whole sky in visible light images.

`astroplan` provides a means to obtain these images and plot them.  It uses `astroquery`, another library, to query the `SkyView` image cutout service.

In [ ]:
from astroplan.plots import plot_finder_image

In [ ]:
ax, hdu = plot_finder_image(pcyg_target, survey='DSS', fov_radius=25*u.arcmin)

In [ ]:
ax, hdu = plot_finder_image(polaris_target, survey='DSS', fov_radius=25*u.arcmin)

In [ ]:
ax, hdu = plot_finder_image(rhocas_target, survey='DSS', fov_radius=25*u.arcmin)

In [ ]:
ax, hdu = plot_finder_image(hd168607_target, survey='DSS', fov_radius=25*u.arcmin)

You might notice these images come out a little smaller than you would like.  You can control the figure size using matplotlib commands:

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
# specify the figure size in inches
pl.figure(figsize=(15, 15))
# a "reticle" is a set of crosshairs highlighting the target source
ax, hdu = plot_finder_image(hd168607_target, survey='DSS', fov_radius=25*u.arcmin, reticle=True)

## Planning for Clouds

What if the night is cloudy?  For professional observations, we will often write off cloudy nights as a lost cause, but for this class, sometimes we need to find the holes in the clouds and observe whatever's there! 

We can enable this by making sky charts.

In [ ]:
from astroplan.plots import plot_sky

In [ ]:
# first, let's find out where the objects are at sunset
sunset = CTO.sun_set_time(now)
# then, let's look 1 hour after sunset
onehour_after_sunset = sunset + 1*u.hour
twohours_after_sunset = sunset + 2*u.hour

Note 1/16/2021: There is a bug in astroplan that will happen if you have matplotlib>=3.3.  


Try the code below. If it fails with [this error](https://github.com/astropy/astroplan/issues/468) about "`ValueError: The number of FixedLocator locations (8), usually from a call to set_ticks, does not match the number of ticklabels (7).`", then do:

`%pip install https://github.com/keflavich/astroplan/archive/fix_plot_sky.zip`

in its own cell and then re-start the notebook and run all cells above this one.  You can do that with the "Restart & Run All" button in the "Kernel" menu above.

In [ ]:
targets = [hd168607_target, pcyg_target, rhocas_target, polaris_target]
plot_sky(target=targets, observer=CTO, time=sunset)
plot_sky(target=targets, observer=CTO, time=onehour_after_sunset)
plot_sky(target=targets, observer=CTO, time=twohours_after_sunset)

In the star chart, the colors represent different times.  One of the points only shows the final color (maybe you can see some others behind it).  Why isn't it moving?

A:

# EXERCISE

Create an altitude plot (showing the 'up time'), a sky chart, and a finder chart for each of the following sources:
1. M33
2. NGC 869
3. Gamma Andromedae